In [1]:
%matplotlib notebook

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import os

pd.set_option('display.max_rows', 500)

In [3]:
covid_df = pd.read_csv(os.path.join("COVID","03-16-2020.csv"))
world_pop_df = pd.read_csv(os.path.join('GDP', 'world_countries_gdp.csv'))
median_age_df = pd.read_csv(os.path.join('Median Age', 'output','median_age.csv'))

#number of countries in each csv
print(f"COVID-19 Countries: {covid_df['Country/Region'].nunique()}")
print(f"World Population Countries: {world_pop_df['country'].nunique()}")
print(f"Median Age Countries: {median_age_df['Country'].nunique()}")
#turn last update to datetime
#to_datetime method. exclude time. just keep date. use map - string slicing method

COVID-19 Countries: 156
World Population Countries: 211
Median Age Countries: 230


In [4]:
#get date only
#map takes in a function. So we use a temporary function (lambda)
covid_df['Last Update'] = covid_df['Last Update'].map(lambda x: x[0:10])

In [5]:
#clean data
grouped_covid_df = covid_df.groupby(['Country/Region']).sum()
grouped_covid_df.drop(['Latitude','Longitude'],1,inplace=True)
grouped_covid_df.rename(index = {"US":"United States","Taiwan*":"Taiwan"}, inplace=True)
grouped_covid_df.reset_index(inplace=True)
grouped_covid_df.rename(columns = {"Country/Region":"Country"}, inplace=True)
#grouped_covid_df.query('Country_Region.str.contains("Congo")',engine='python',inplace=True)
grouped_covid_df

,Country,Confirmed,Deaths,Recovered
0,Afghanistan,21,0,1
1,Albania,51,1,0
2,Algeria,54,4,12
3,Andorra,2,0,1
4,Antigua and Barbuda,1,0,0
5,Argentina,56,2,1
6,Armenia,52,0,0
7,Aruba,2,0,0
8,Australia,377,3,23
9,Austria,1018,3,6


In [20]:
#clean data
grouped_world_pop_df = world_pop_df.groupby('country').sum()
grouped_world_pop_df.drop(['rank','unGDP'],1,inplace=True)
grouped_world_pop_df.rename(index = {"South Korea":"Korea, South","Bahamas":"The Bahamas"}, 
                            columns = {"imfGDP": "GDP","gdpPerCapita": "GDP Per Capita","pop":"Population"},
                            inplace=True)
grouped_world_pop_df.index.name = 'Country'
grouped_world_pop_df.reset_index(inplace=True)
grouped_world_pop_df

,Country,GDP,GDP Per Capita,Population
0,Afghanistan,2.068200e+10,531.2838,38928.346
1,Albania,1.721000e+10,5980.2689,2877.797
2,Algeria,1.930560e+11,4402.5406,43851.044
3,Andorra,0.000000e+00,36994.8441,77.265
4,Angola,9.642600e+10,2933.8892,32866.272
5,Anguilla,0.000000e+00,22496.8653,15.003
6,Antigua and Barbuda,1.809000e+09,18472.5669,97.929
7,Argentina,5.153530e+11,11402.6811,45195.774
8,Armenia,1.386800e+10,4680.0077,2963.243
9,Aruba,2.952000e+09,27649.2516,106.766


In [21]:
clean_median_age_df = median_age_df.copy()
#replace country column with string replace
clean_median_age_df['Country'] = clean_median_age_df['Country'].str.replace(pat='And',repl='and',case=False)
clean_median_age_df['Country'] = clean_median_age_df['Country'].str.replace(pat='The',repl='the',case=False)
clean_median_age_df['Country'] = clean_median_age_df['Country'].str.replace(pat='Congo, Democratic Republic Of the',repl='Republic of the Congo',case=False)
clean_median_age_df['Country'] = clean_median_age_df['Country'].str.replace(pat='andorra',repl='Andorra',case=False)
clean_median_age_df['Country'] = clean_median_age_df['Country'].str.replace(pat='Bahamas, the',repl='The Bahamas',case=False)
clean_median_age_df.sort_values(by='Country',ascending=True,inplace=True)
clean_median_age_df.reset_index(drop=True,inplace=True)
clean_median_age_df

,Country,Median Age
0,Afghanistan,19.0
1,Albania,33.4
2,Algeria,28.3
3,American Samoa,26.1
4,Andorra,44.9
5,Angola,15.9
6,Anguilla,35.1
7,Antigua and Barbuda,32.2
8,Argentina,31.9
9,Armenia,35.6


In [22]:
merged_df = grouped_covid_df.merge(grouped_world_pop_df,how='outer',on='Country')
#merged_df.dropna(inplace=True)
merged_df.reset_index(drop=True,inplace=True)
#merged_df['GDP Per Capita'] = merged_df['GDP Per Capita'].map('{:.0f}'.format)
#merged_df['Population'] = (merged_df['Population'] * 1000).astype('int64')

In [25]:
complete_df = merged_df.merge(clean_median_age_df,how='outer',on='Country')
complete_df.sort_values(by='Country',ascending=True,inplace=True)
complete_df.reset_index(drop=True,inplace=True)
#complete_df.dropna(inplace=True)
#complete_df.reset_index(drop=True,inplace=True)

df1 = complete_df[complete_df.isna().any(axis=1)]
df1
complete_df

,Country,Confirmed,Deaths,Recovered,GDP,GDP Per Capita,Population,Median Age
0,Afghanistan,21.0,0.0,1.0,2.068200e+10,531.2838,38928.346,19.0
1,Albania,51.0,1.0,0.0,1.721000e+10,5980.2689,2877.797,33.4
2,Algeria,54.0,4.0,12.0,1.930560e+11,4402.5406,43851.044,28.3
3,American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,26.1
4,Andorra,2.0,0.0,1.0,0.000000e+00,36994.8441,77.265,44.9
5,Angola,NaN,NaN,NaN,9.642600e+10,2933.8892,32866.272,15.9
6,Anguilla,NaN,NaN,NaN,0.000000e+00,22496.8653,15.003,35.1
7,Antigua and Barbuda,1.0,0.0,0.0,1.809000e+09,18472.5669,97.929,32.2
8,Argentina,56.0,2.0,1.0,5.153530e+11,11402.6811,45195.774,31.9
9,Armenia,52.0,0.0,0.0,1.386800e+10,4680.0077,2963.243,35.6


In [26]:
complete_df.to_csv('output\complete_data.csv', index=False)